<a href="https://colab.research.google.com/github/aetev/Hearth-Stone-Python-Simulator/blob/main/card_enc.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [11]:
import json
import tensorflow as tf
import re
# Assuming AtomicCards.json is in your Google Drive, adjust the path if necessary
file_path = '/content/drive/MyDrive/MTGdata/AtomicCards.json'


with open(file_path, 'r') as f:
  data = json.load(f)['data']

In [6]:
def filter_legal_cards(data):
    legal_commander_data = {}
    for card_name, card_data in data.items():
        try:
            if card_data[0]['legalities']['commander'] == 'Legal':
                legal_commander_data[card_name] = card_data[0] # Store only the first element of card_data
        except (KeyError, IndexError):
            pass  # Or print a message, log it, etc.
    return legal_commander_data

filtered_data = filter_legal_cards(data)


In [35]:
items = list(filtered_data.items())

second_key, second_value = items[11]

text = second_value['text']
print(text)

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)


In [53]:
class TextCleaningLayer(tf.keras.layers.Layer):
    def __init__(self, **kwargs):
        super(TextCleaningLayer, self).__init__(**kwargs)

    def call(self, inputs):
        # Convert input to string if it's not already
        if not isinstance(inputs, tf.Tensor):
           inputs = tf.convert_to_tensor(inputs, dtype=tf.string)

        # 1. Lowercase the text
        cleaned_text = tf.strings.lower(inputs)

        # 2. Remove special characters (except for spaces + - /)

        cleaned_text = tf.strings.regex_replace(cleaned_text, '\.', ' ')

        cleaned_text = tf.strings.regex_replace(cleaned_text, '\([^)]*\)', '')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '\{\}', ' ')
        cleaned_text = tf.strings.regex_replace(cleaned_text, '[^\w\s+/\-]', '')

        cleaned_text = tf.strings.regex_replace(cleaned_text, '\n', '')

        return cleaned_text

cleaned_text = TextCleaningLayer()
print(text)
print(cleaned_text.call(text))

Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)
tf.Tensor(b'up to two target creatures each get +1/+0 and gain first strike until end of turn  madness 1r ', shape=(), dtype=string)


In [56]:
# prompt: use regex to relace "{}" with white space

import re



# Use regex to replace "{}" with whitespace
cleaned_text = re.sub(r"\{\}", " ", text)

print(cleaned_text)


Up to two target creatures each get +1/+0 and gain first strike until end of turn. (They deal combat damage before creatures without first strike.)
Madness {1}{R} (If you discard this card, discard it into exile. When you do, cast it for its madness cost or put it into your graveyard.)
